# 暗号技術入門

## 古典暗号を C# で実装してみる

### 準備

C# のコードを追加します。

In [1]:
Console.WriteLine("ここから始めます")

The below script needs to be able to find the current output cell; this is an easy method to get it.

ここから始めます


## 2-1 古典暗号の概要

平文を暗号空間に 1:1 対応させる

### 文字の符号化

空白文字や記号をどう暗号化する？

コンピューターの世界では文字は共通の符号化規則がある。

例: ASCII など

### Tips 日本語の暗号?

日本語を符号化する。様々な符号化方式があるが、UTF8 が現在ではデファクト。

In [9]:
var str = "ここから始めます";
Console.WriteLine(str);

ここから始めます


In [125]:
var encodedBytes = Encoding.UTF8.GetBytes(str);

foreach(byte b in encodedBytes)
{
    //.Net では byte は 0-255 の数字で表される
    Console.WriteLine(b);
}

Console.WriteLine($"{encodedBytes.Length} 個のバイト");

227
129
147
227
129
147
227
129
139
227
130
137
229
167
139
227
130
129
227
129
190
227
129
153
24 個のバイト


この byte の羅列を UTF8 という符号化方式で文字に変換すると "ここから始めます" になる。
UTF8 の場合、日本語文字は 1 文字が 3byte ~ (可変だったはず) であらわされる。 
ちなみに符号化された文字は、よく HEX (16進数) で表されたりする。

In [27]:
var encodedHex = BitConverter.ToString(encodedBytes);

Console.WriteLine(str);
Console.WriteLine(encodedHex);


ここから始めます
E3-81-93-E3-81-93-E3-81-8B-E3-82-89-E5-A7-8B-E3-82-81-E3-81-BE-E3-81-99


### 2-2 シーザー暗号

シーザー暗号とは、文字をずらず暗号

たとえば、元のアルファベットから 3 つずらした数字が暗号だとすると、このような表になる。

|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|A|B|C|


In [16]:
//小文字の平文
var plainText = "helloworld";

//秘密鍵
var priKey = 3;

//バイト配列に変換
var bytes = Encoding.ASCII.GetBytes(plainText);

byte[] encryptedText = new byte[bytes.Length];

for(var i = 0; i < bytes.Length; i++)
{
    //a-z を ACII 表現すると 97~122
    //a-z を 0-25 の文字列で表現
    var c = bytes[i]-97;

    //priKey だけずらして、97足すと元の場所に
    var enc = (c+ priKey)%26;
    if(enc < 0)
    {
        enc += 26;
    }
    enc += 97;
    encryptedText[i] = (byte)enc;
}

//バイト配列からデコード
var encrypted = System.Text.Encoding.ASCII.GetString(encryptedText);

Console.WriteLine(encrypted.ToUpper());

KHOORZRUOG


In [126]:
//復号は逆に戻すだけ
var plainText = "KHOORZRUOG";

//マイナスにすれば OK
var priKey = -3;

//小文字にしてからバイト配列に変換
var bytes = Encoding.ASCII.GetBytes(plainText.ToLower());

byte[] encryptedText = new byte[bytes.Length];

for(var i = 0; i < bytes.Length; i++)
{
    //a-z を ACII 表現すると 97~122
    //a-z を 0-25 の文字列で表現
    var c = bytes[i]-97;

    //priKey だけずらして、97足すと元の場所に
    var enc = (c+ priKey)%26;
    if(enc < 0)
    {
        enc += 26;
    }
    enc += 97;
    encryptedText[i] = (byte)enc;
}

//バイト配列からデコード
var encrypted = System.Text.Encoding.ASCII.GetString(encryptedText);

Console.WriteLine(encrypted);

helloworld


クラスにしてみる。

In [17]:
using System;
public static class ROT
{
  public static string Encrypt(string plainText, int priKey)
  {
    if(plainText == null)
    {
        throw new ArgumentNullException("plainText should not be null");
    }
    var bytes = Encoding.ASCII.GetBytes(plainText);

    byte[] encryptedText = new byte[bytes.Length];

    for(var i = 0; i < bytes.Length; i++)
    {
        if(bytes[i] < 97 || 122 <bytes[i])
        {
            //小文字のアルファベット以外は変換しない
            encryptedText[i] = bytes[i];
        } else {
            //a-z を ACII 表現すると 97~122
            //a-z を 0-25 の文字列で表現
            var c = bytes[i]-97;

            //priKey だけずらして、97足すと元の場所に
            var enc = (c+ priKey)%26;
            if(enc < 0)
            {
                enc += 26;
            }
            enc += 97;
            encryptedText[i] = (byte)enc;
        }
    }
    //バイト配列からデコード
    var encrypted = System.Text.Encoding.ASCII.GetString(encryptedText);
    return encrypted.ToUpper();
  }
  public static string Decrypt(string encrypted, int key)
  {
      return ROT.Encrypt(encrypted.ToLower(), -key).ToLower();
  }
}

var enc = ROT.Encrypt("hello world", 3);
Console.WriteLine(enc);
var dec = ROT.Decrypt(enc, 3);
Console.WriteLine(dec);


KHOOR ZRUOG
hello world


#### 問題: KZDVWCZVJCZBVRERIFN を解読する

In [19]:
var encrypted = "KZDVWCZVJCZBVRERIFN";

Console.WriteLine(-1%26);
for(var i=0; i < 26; i++)
{
    Console.WriteLine($"{i} {ROT.Decrypt(encrypted,i)}");
}

-1
0 kzdvwczvjczbvrerifn
1 jycuvbyuibyauqdqhem
2 ixbtuaxthaxztpcpgdl
3 hwastzwsgzwysobofck
4 gvzrsyvrfyvxrnanebj
5 fuyqrxuqexuwqmzmdai
6 etxpqwtpdwtvplylczh
7 dswopvsocvsuokxkbyg
8 crvnournburtnjwjaxf
9 bqumntqmatqsmivizwe
10 aptlmsplzsprlhuhyvd
11 zosklrokyroqkgtgxuc
12 ynrjkqnjxqnpjfsfwtb
13 xmqijpmiwpmoierevsa
14 wlphiolhvolnhdqdurz
15 vkoghnkgunkmgcpctqy
16 ujnfgmjftmjlfbobspx
17 timeflieslikeanarow
18 shldekhdrkhjdzmzqnv
19 rgkcdjgcqjgicylypmu
20 qfjbcifbpifhbxkxolt
21 peiabheaohegawjwnks
22 odhzagdzngdfzvivmjr
23 ncgyzfcymfceyuhuliq
24 mbfxyebxlebdxtgtkhp
25 laewxdawkdacwsfsjgo


### 2-4 スキュタレー暗号

帯に巻き付けた紙に、別の角度から書き込むことで暗号化

紙を巻き付けて別の角度から読むことで、復号する

In [79]:
using System.Text;

Console.WriteLine($"{plainText}  //平文");

//暗号用の円周の長さ
var c = 4;

//パディングの長さ
var paddingLength = plainText.Length % c;

const string alphabets = "abcdefghijklmnopqrstuvwxyz";

static public string GenerateRandomAlphabets(int length)
{
    var r = new Random();
    var sb = new StringBuilder();
    for(var i = 0; i < length; i++){
       sb.Append(alphabets[r.Next(alphabets.Length)]); 
    }
    return sb.ToString();
}

var padding = GenerateRandomAlphabets(paddingLength);

var plainTextPadding = plainText + padding;

Console.WriteLine($"{plainTextPadding}  //パディングされた平文");

//横に3回読む
var encrypted = new StringBuilder();
for(var i = 0; i < c; i++)
{
    //3つ飛ばしで文字を出力する
    for(var j = 0; j + i < plainTextPadding.Length; j+=c)
    {
        var c = plainTextPadding[j+i];
        encrypted.Append(c);
    }
}

Console.WriteLine($"{encrypted.ToString()}  //暗号文");

//復号は文字の長さを円柱で割った数分とばして読む
var encyptedLineLength = encrypted.Length / c;

var decrypted = new StringBuilder();
for(var i = 0; i < encyptedLineLength; i++)
{
    for(var j = 0; j + i < plainTextPadding.Length; j+=encyptedLineLength)
    {
        var c = encrypted[j+i];
        decrypted.Append(c);
    }
}

Console.WriteLine($"{decrypted.ToString()}  //復号");




akademeia  //平文
akademeiab  //パディングされた平文
aeakmbaedi  //暗号文
aamadekbei  //復号


### 2-5 転置式暗号

スキュレター暗号は、文字を並び替える暗号。より一般化すると、転地式暗号と呼ばれる。
あるブロックごとに分けられた文字列を、並び替えて暗号文にする。

ナポレオンの暗号文

In [73]:
var plainText = "akademeia";
var encrypted = new StringBuilder();
for(var i = 0; i < plainText.Length; i += 2)
{
    if(i + 1 < plainText.Length)
    {
        encrypted.Append(plainText[i+1]);
    }
    encrypted.Append(plainText[i]);
}

Console.WriteLine($"{encrypted.ToString()}  //暗号文")


kadameiea  //暗号文


### 2-6 単一換字式暗号

文字を全く違い記号や文字に置換して変換する。表したい文字と同じだけの長さの暗号鍵が必要。

In [124]:
var plainText = "akademeia";

const string alphabets = "abcdefghijklmnopqrstuvwxyz";
const string sigma = "dicoxvutpayqlhbgswrfzmenkj";

var encrypted = new StringBuilder();
for(var i = 0; i < plainText.Length; i++)
{
    var num = Array.FindIndex(alphabets.ToArray(), c => c == plainText[i]);
   encrypted.Append(sigma[num]);
}

Console.WriteLine($"{encrypted.ToString()} //暗号文");

//戻すには同じ暗号かぎで逆操作する。

var encryptedText = encrypted.ToString();

var decrypted = new StringBuilder();
for(var i = 0; i < encryptedText.Length; i++)
{
    var num = Array.FindIndex(sigma.ToArray(), c => c == encryptedText[i]);
    decrypted.Append(alphabets[num]);
}

Console.WriteLine($"{decrypted.ToString()} //復号");

dydoxlxpd //暗号文
akademeia //復号
